<div class="caja" style ="font-family: sans-serif;
    font-size: 13px;
    font-weight: 400;
	/*font-weight: bold;*/
    color: #fff;
    /*background: #889ccf;*/
    margin: 0 0 25px;
    overflow: hidden;
    padding: 20px;
    order: 10px solid #025280;
    background: #0F80C1;
    border-radius: 4px;
    border-left: 3px solid #025280;
    border-bottom: 2px solid #025280;
    border-right: 1px solid #025280;
    border-top: 0.5px solid #025280;
    height: 35px;
    padding: 10px;">UCSP- DIP en Machine Learning, Deep Learning y sus Aplicaciones en Industria | Course 2: Modelos de Regresión, Clasificación y Agrupamiento | Profesor: Jose Ocho Luna |  Presentado: Hector Pérez V.</div>

## Filtros Spam con Naive Bayes

## Explorando Dataset


In [1]:
import pandas as pd

sms_spam = pd.read_csv('data/SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


A continuación, vemos que alrededor del 87% de los mensajes son `ham` y el 13% restante son `spam`. Esta muestra parece representativa, ya que en la práctica la mayoría de los mensajes que recibe la gente son `ham`.

In [2]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Conjunto de entrenamiento y prueba

Conjunto de entrenamiento representa el 80% de los datos y el conjunto de prueba el 20% restante.

In [3]:
# Conjunto de datos alearorio
data_randomized = sms_spam.sample(frac=1, random_state=1)

# Calcular el indice par split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


Esperamos que los porcentajes se acerquen a lo que tenemos en el conjunto de datos completo, donde alrededor del 87% de los mensajes son `ham` y el 13% restante son `spam`

In [4]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [5]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

## Limpieza de Datos
limpieza de datos para traer los datos en un formato que nos permita extraer fácilmente toda la información que necesitamos.

Básicamente, queremos llevar los datos a este formato:

img

### Letras Mayusculas  y minusculas  y signos  de  puntuación


In [6]:
# Antes de la limpieza de datos
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
# Despues de la limpieza de datos
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

C:\Users\HECTOR\AppData\Local\Temp/ipykernel_15476/3036846089.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creando el vocabulario
Creación del vocabulario, que en este contexto significa una lista con todas las palabras únicas en nuestro conjunto de entrenamiento

In [8]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

7783 palabras únicas en todos los mensajes  de nuestro data set

In [9]:
len(vocabulary)

7783

### El conjunto de entrenamiento final

Ahora vamos a utilizar el vocabulario que acabamos de crear para realizar la transformación de datos que queremos

In [10]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,bruce,wishes,ive,on,gonnamissu,shared,ends,imprtant,cricket,luks,...,scotland,teacher,instructions,unspoken,06,praises,totes,challenging,31p,tigress
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,bruce,wishes,ive,on,gonnamissu,shared,ends,imprtant,...,scotland,teacher,instructions,unspoken,06,praises,totes,challenging,31p,tigress
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##  Cálcular primero las constantes

Ahora hemos terminado con la limpieza del dataset, creamos el filtro de correo no deseado. 

El algoritmo Naive Bayes deberá responder estas dos preguntas para poder clasificar nuevos mensajes:

$$

P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$

$$

P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)

$$

Además, para calcular P(wi|Spam) y P (wi|Ham) dentro de las fórmulas anteriores, necesitaremos usar estas ecuaciones:
$$

P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$

$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
A continuación, usaremos nuestro conjunto de entrenamiento para calcular:
* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary

Usaremos Laplace  $\alpha = 1$.

In [13]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Cálculo de parámetros

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

Ahora que tenemos los términos constantes calculados anteriormente, podemos continuar con el cálculo de los parámetros $P(w_i|Spam)$ y $P(w_i|Ham)$ Por tanto, cada parámetro será un valor de probabilidad condicional asociado con cada palabra del vocabulario.

Los parámetros se calculan mediante las fórmulas:

$$

P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$

$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [14]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Clasificando un mensaje nuevo

Ahora que tenemos todos nuestros parámetros calculados, podemos empezar a crear el filtro de spam. El filtro de spam puede entenderse como una función que:

 * Toma como entrada un nuevo mensaje (w1, w2, ..., wn).
 * CalculaP(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
 * Compara los valores de P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), y:
  * If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then el mensaje es classificado como ham.
  * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then el mensaje es classificado como.
  * If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then  el algoritmo puede solicitar human help.

In [15]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [16]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [17]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Medición de la precisión del filtro de spam

Los dos resultados anteriores parecen prometedores, pero veamos qué tan bien funciona el filtro en nuestro conjunto de prueba, que tiene 1114 mensajes.

Comenzaremos escribiendo una función que devuelva etiquetas de clasificación en lugar de imprimirlas.

In [18]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Ahora que tenemos una función que devuelve etiquetas en lugar de imprimirlas, podemos usarla para crear una nueva columna en nuestro conjunto de prueba.

In [19]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Ahora, escribiremos una función para medir la precisión de nuestro filtro de correo no deseado y averiguar qué tan bien lo hace nuestro filtro de correo no deseado.

In [20]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


No ha visto en los entrenamientos y ha clasificado 1.100 correctamente

